In [96]:
import pandas as pd
from datetime import datetime


In [97]:
pd.set_option('display.max_rows', None)

In [98]:
ETF = pd.read_csv("vanguard_s&p500.csv", index_col=0, parse_dates=True)


In [99]:
ETF.isna().sum()

aroon_Aroon Down                                  25
aroon_Aroon Up                                    25
bbands_close_Real Lower Band                      19
bbands_close_Real Middle Band                     19
bbands_close_Real Upper Band                      19
cci_CCI                                           13
ema_close_20_EMA                                  19
ema_close_50_EMA                                  49
macd_close_MACD                                   33
macd_close_MACD_Hist                              33
macd_close_MACD_Signal                            33
obv_OBV                                            0
rsi_close_RSI                                     14
stoch_SlowD                                        8
stoch_SlowK                                        8
stock_time_series_adjusted_open                    0
stock_time_series_adjusted_high                    0
stock_time_series_adjusted_low                     0
stock_time_series_adjusted_close              

In [122]:
# There are couple of columns that have way too many NA values and have to be deleted
cols_to_del_NA = ["crudeoil_Change", "dollar_Wave", "dollar_EFP Volume", "dollar_EFS Volume", "dollar_Block Volume",
             "fedfund_Change", "fedfund_Last", "gold_Change", "tenyrnote_Last", "tenyrnote_Change", "treasury_2 MO"]

# For treasury notes, I will only keep 3-month treasury notes and 10-year treasury notes 
# This is to identify presence of yield curve inversion
cols_to_del_treasury = ["treasury_1 MO", "treasury_6 MO", "treasury_1 YR", "treasury_2 YR", "treasury_3 YR",
                       "treasury_5 YR", "treasury_7 YR", "treasury_20 YR", "treasury_30 YR"]

# For the S&P 500 ETF, we will only look at the close price, getting rid of other variables associated to the ETF
cols_to_del_SP500 = ["stock_time_series_adjusted_open", "stock_time_series_adjusted_high", "stock_time_series_adjusted_low",
                    "stock_time_series_adjusted_close", "stock_time_series_adjusted_volume", "stock_time_series_adjusted_split_coefficient",
                    "stock_symbol", "stock_time_series_adjusted_dividend_amount"]

cols_to_del_MACD = ["macd_close_MACD", "macd_close_MACD_Signal"]

# I will take the slow K STOCH value instead of the fast K STOCH to reduce sensitivity
cols_to_del_stoch = ["stoch_SlowK"]

cols_to_del_crudeoil = ["crudeoil_Last", "crudeoil_High", "crudeoil_Low", "crudeoil_Change", "crudeoil_Settle",
                       "crudeoil_Volume", "crudeoil_Previous Day Open Interest"]

cols_to_del_fedfund = ["fedfund_High", "fedfund_Low", "fedfund_Last", "fedfund_Change", "fedfund_Settle",
                      "fedfund_Volume", "fedfund_Previous Day Open Interest"]

cols_to_del_dollar = ["dollar_High", "dollar_Low", "dollar_Settle", "dollar_Change", "dollar_Volume", 
                      "dollar_Prev. Day Open Interest"]

cols_to_del_gold = ["gold_High", "gold_Low", "gold_Last", "gold_Change", "gold_Settle", "gold_Volume",
                   "gold_Previous Day Open Interest"]

total_cols_to_del = list(set(cols_to_del_NA + cols_to_del_treasury + cols_to_del_SP500 + cols_to_del_MACD + 
                            cols_to_del_stoch + cols_to_del_crudeoil + cols_to_del_fedfund + cols_to_del_gold + 
                             cols_to_del_dollar))

remaining_cols = [column for column in ETF.columns if column not in total_cols_to_del]

In [123]:
ETF_fewer_cols = ETF[remaining_cols]

In [124]:
ETF_fewer_cols.columns

Index(['aroon_Aroon Down', 'aroon_Aroon Up', 'bbands_close_Real Lower Band',
       'bbands_close_Real Middle Band', 'bbands_close_Real Upper Band',
       'cci_CCI', 'ema_close_20_EMA', 'ema_close_50_EMA',
       'macd_close_MACD_Hist', 'obv_OBV', 'rsi_close_RSI', 'stoch_SlowD',
       'stock_time_series_adjusted_adjusted_close',
       'corporatebond_BAMLC0A0CMEY', 'crudeoil_Open', 'dollar_Open',
       'fedfund_Open', 'gold_Open', 'treasury_3 MO', 'treasury_10 YR'],
      dtype='object')

In [125]:
# For treasury notes, I will only keep 3-month treasury notes and 10-year treasury notes 
# This is to identify presence of yield curve inversion


In [126]:
ETF_fewer_cols.isna().sum()

aroon_Aroon Down                             25
aroon_Aroon Up                               25
bbands_close_Real Lower Band                 19
bbands_close_Real Middle Band                19
bbands_close_Real Upper Band                 19
cci_CCI                                      13
ema_close_20_EMA                             19
ema_close_50_EMA                             49
macd_close_MACD_Hist                         33
obv_OBV                                       0
rsi_close_RSI                                14
stoch_SlowD                                   8
stock_time_series_adjusted_adjusted_close     0
corporatebond_BAMLC0A0CMEY                    4
crudeoil_Open                                 4
dollar_Open                                   4
fedfund_Open                                  6
gold_Open                                     6
treasury_3 MO                                19
treasury_10 YR                               19
dtype: int64

In [127]:
len(ETF_fewer_cols.columns)

20

In [81]:
ETF_fewer_cols.loc['2019-11', :]["treasury_3 MO"]

2019-11-29    1.59
2019-11-27    1.62
2019-11-26    1.60
2019-11-25    1.61
2019-11-22    1.58
2019-11-21    1.58
2019-11-20    1.57
2019-11-19    1.57
2019-11-18    1.57
2019-11-15    1.57
2019-11-14    1.57
2019-11-13    1.57
2019-11-12    1.59
2019-11-11     NaN
2019-11-08    1.55
2019-11-07    1.56
2019-11-06    1.56
2019-11-05    1.56
2019-11-04    1.53
2019-11-01    1.52
Name: treasury_3 MO, dtype: float64

In [52]:
ETF_fewer_cols[ETF_fewer_cols["treasury_1 MO"].isna()==True]

,aroon_Aroon Down,aroon_Aroon Up,bbands_close_Real Lower Band,bbands_close_Real Middle Band,bbands_close_Real Upper Band,cci_CCI,ema_close_200_EMA,ema_close_20_EMA,macd_close_MACD,macd_close_MACD_Hist,...,treasury_3 MO,treasury_6 MO,treasury_1 YR,treasury_2 YR,treasury_3 YR,treasury_5 YR,treasury_7 YR,treasury_10 YR,treasury_20 YR,treasury_30 YR
2019-11-11,4.0,92.0,271.5955,278.4285,285.2615,82.1704,266.8528,278.9378,2.9257,0.4673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-14,72.0,32.0,264.2558,271.5035,278.7512,57.2186,264.2474,270.5097,-0.5373,-0.0389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-12,60.0,4.0,241.9922,251.3640,260.7358,32.3199,252.9621,253.1705,-1.3732,1.1215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-08,100.0,56.0,264.2760,267.1415,270.0070,-186.9693,253.2225,266.4917,0.3895,-0.5980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-09,4.0,100.0,227.4882,230.6830,233.8778,104.4613,219.7867,230.7175,1.6039,0.2826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-11,80.0,96.0,191.6346,195.7735,199.9124,114.0484,193.3353,196.1914,0.1209,0.6104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-10,28.0,8.0,194.7156,197.3155,199.9154,73.4836,192.6689,197.9415,-0.1135,0.1049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-11,0.0,76.0,184.5626,189.8535,195.1444,-33.3348,186.4483,189.5286,2.2328,-0.2865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-12,64.0,32.0,172.1045,179.6550,187.2055,107.2113,185.7185,180.6599,0.4886,1.2400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-11,24.0,100.0,170.1376,181.0005,191.8634,85.2819,170.3258,182.9251,2.2899,0.7633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It seems that the NA values for ten year treasury notes values occur at the same dates